Goal is to extract the questions from the paper networks in a same field and show case the strucutre of questions (with their frequency)

In [1]:
import networkx as nx 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json 
import pickle
import os

from IPython.core.debugger import set_trace


In [2]:
def getQid(qtext,Qdic):
    qtext = str(qtext)
    if qtext in Qdic['label']:
        idx = Qdic['label'].index(qtext)
        if paper not in Qdic['paper'][idx]:
            Qdic['paper'][idx].append(paper)
        return Qdic['id'][idx]
    else:
        newid = 'Q%d'%np.random.randint(100000)
        Qdic['id'].append(newid)
        Qdic['label'].append(qtext)
        Qdic['paper'].append([paper])
        return newid
def addedge(qids,etext,Edgedic):
    if qids in Edgedic['startNend']:
        idx = Edgedic['startNend'].index(qids)
        if paper not in Edgedic['paper'][idx]:
            Edgedic['paper'][idx].append(paper)
        if etext not in Edgedic['label'][idx]:
            Edgedic['label'][idx].append(etext)
    else:
        Edgedic['startNend'].append(qids)
        Edgedic['id'].append('E%d'%np.random.randint(100000))
        Edgedic['label'].append([etext])
        Edgedic['paper'].append([paper])

In [3]:
papers = ['Rich2019_the']#['Burda2019_Large','Houthooft2016_VIME','Pathak2019_Self-Supervised']
fdir = '../database/curiosity/compcogsci/'
#TODO: load the paper entity list instead of just streams. "paper" should all point to there.
Qdic = {'id':[],'label':[],'paper':[]}
Qedgs = {'startNend':[],'label':[],'id':[],'paper':[]}
for paper in papers:
    [nodedf,edgedf,allngrps] = pickle.load(open(fdir+paper+'_qnet.p','rb'))
    for row in edgedf.itertuples(index=False):
        if row.end[0]=='q':
            if row.start[0]=='q': # q to q link
                oldids = [row.start,row.end]
                lbs = [nodedf['label'].loc[nodedf['id']==oldids[0]].values[0],nodedf['label'].loc[nodedf['id']==oldids[1]].values[0]]
                Qids = [getQid(lbs[0],Qdic),getQid(lbs[1],Qdic)]
            elif (row.end[0]=='q' and row.start[0]=='a'): # a to q
                prevqid = edgedf['start'].loc[edgedf['end']==row.start].values[0]
                if prevqid[0]!='q':
                    print('strange! Answer not following a question!')
                    set_trace()
                    continue
                prevqlb = nodedf['label'].loc[nodedf['id']==prevqid].values[0]
                thislb = nodedf['label'].loc[nodedf['id']==row.end].values[0]
                Qids = [getQid(prevqlb,Qdic),getQid(thislb,Qdic)]
            elb = row.label
            addedge(Qids,elb,Qedgs)


In [4]:
Qdf = pd.DataFrame(Qdic)
Qedgedf = pd.DataFrame(Qedgs)
pickle.dump([Qdf,Qedgedf],open(fdir+papers[0]+'_Qonly.p','wb'))#'combQnet.p','wb'))

# alternatively, generate pure Q network

In [5]:
papers = ['Dezza2017_Learning']#['Burda2019_Large','Houthooft2016_VIME','Pathak2019_Self-Supervised']
fdir = '../database/curiosity/compcogsci/'
#TODO: load the paper entity list instead of just streams. "paper" should all point to there.
Qedgs =[]
for paper in papers:
    [nodedf,edgedf,allngrps] = pickle.load(open(fdir+paper+'_qnet.p','rb'))
    for row in edgedf.itertuples(index=False):
        if row.end[0]=='q':
            thisid = row.end
            if (row.end[0]=='q' and row.start[0]=='a'): # a to q
                prevqid = edgedf['start'].loc[edgedf['end']==row.start].values[0]
                eid = 'l%d'%np.random.randint(100000)
                if prevqid[0]!='q':
                    print('strange! Answer not following a question!')
                    set_trace()
                    continue
            else:
                prevqid = row.start
                eid = row.id
            elb = row.label
            
            edgedict = {'start':prevqid,'end':thisid,'label':elb,'id':eid}
            Qedgs.append(edgedict)
Qnodes = []
for node in nodedf.itertuples(index=False):
    if node.id[0]=='q':
        Qnodes.append(node)
Qndf = pd.DataFrame(Qnodes)
Qedf = pd.DataFrame(Qedgs)

In [6]:
pickle.dump([Qndf,Qedf],open(fdir+papers[0]+'_Qonly.p','wb'))